## SICP 习题 （3.17）解题总结: 正确的序对统计方法

SICP 习题 3.17要求我们修改 习题 3.16的序对统计代码，让它正常工作。

关于3.16题中的代码错误我们在3.16的题解中已经做了讨论，现在是要看如何修改。题目也有提示，关键思路就是构建另一个数据结构保存已经遍历的序对就好了。

首先有个问题是我们怎么知道某一个序对已经访问过了？记录一个序对的值是不行的，因为一个列表里可能有两个元素是相同的，他们其实是两个元素，我们不能因为他们的值相等就认为他们是一个东西。

所以这里就涉及到什么是“同一个”的问题。这个看似简单的问题在程序编译，程序解析中都特别重要。

在我们这个上下文了简单给答案的话，可以认为是指向同一段内存的是同一个。

下面通过简单的代码测试一下：

In [3]:
(define leaf (cons 1 2))
(define to-same-leaf (cons leaf leaf))
(eq? (car to-same-leaf) (cdr to-same-leaf))

#t

以上代码里(car to-same-leaf)和(cdr to-same-leaf)认为是同一个，因为他们都指向了leaf，这种“同一性”可以通过eq?来判断。

下面这段代码就可以告诉我们两个“相同”的元素不是“同一个”：

In [2]:
(define leaf-2 (cons 1 2))
(eq? leaf leaf-2)

#f

虽然leaf和leaf-2的值都是(cons 1 2)，但是他们不是同一个东西，简单讲就是修改leaf不会改变leaf-2

### 实现

这个逻辑理顺了要实现就不是那么难了，我们之前做过set这个数据结构，可以通过set来保存已经访问过的序对。

set相关的代码如下，注意这里修改了代码，把equal?换成了eq?，因为我们要比较地址，而不是比较值：

In [22]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
	((eq? x (car set)) #t)
	(else (element-of-set? x (cdr set)))))

(define (adjoin-set x set)
  (if (element-of-set? x set)
      set
      (cons x set)))


简单测试一下：

In [24]:
(define visited-set '())
(set! visited-set (adjoin-set leaf visited-set))

In [25]:
visited-set

((1 . 2))

In [26]:
(element-of-set? leaf visited-set)

#t

In [27]:
(element-of-set? to-same-leaf visited-set)

#f

In [28]:
(element-of-set? leaf-2 visited-set)

#f

下面就是解题的主体，通过一个内部函数count-inner来遍历序对，每遇到一个序对先检查是不是之前遇到过这个序对，如果遇到过的话就跳过这个序对，如果没有遇到过就把这个序对加入到已访问列表里，然后开始遍历它的左树和右树：

In [38]:
(define (count-pairs x)
  (let ((visited-set '()))
    (define (count-inner target)
      (if (not (pair? target))
          0
          (if  (element-of-set? target visited-set)
              0
              (begin 
               (set! visited-set (adjoin-set target visited-set))
               (+ (count-inner (car target))
                 (count-inner (cdr target))
                  1))
              )))
    (count-inner x)
  ))

下面是测试函数：

In [40]:
(define leaf (cons 1 2))
(define to-same-leaf (cons leaf leaf))
(define test-sample (cons to-same-leaf leaf))
(count-pairs test-sample)

3